### Changelog 

- Separating **process_iplist()** output from VT calls to folders (todays_date)
- changed output of **process_json()** FROM "_parsed-combined" to todays_date_parsed-combined in EACH RESPECTIVE FOLDER 

### Flow

1. User edits and feeds ip.csv
2. **process_iplist()** reads each and calls VT api 
3. Responses are stored in **"downloaded_vtresponse"** and seperated into folders by **respective dates** (DDMMYYYY)
4. **process_json()** reads jsons in each folder and generates a compilation for that day in each folder

### Questions / Todo List

**Update** 

I have managed to get the scripts running from jupyter notebook but have the following questions

(a) Receiving (via a web interface) either an individual or list of domais / IP addresses --> currently it's fed via CSV, should i create a front-end for people to upload their files?

(b) Storing list of domains / IP address into a queue based list --> Is this the back-end of things? That is to say, this script runs in the back-end and whenever files come in from front-end it'll trigger the script?

(c) Carryout enrichment --> Where does this "processed in the previous X days" come from? From what i understand, should i create a check such that when new information comes in, it will look at previous histories when the IP/Domain was checked, and continue / stop accordingly? 

(d) storing responses in disk and extracting subset into DB --> is there a specific subset you'd like? DB-wise I would prefer to try NoSQL as i have no experience with it!

In [2]:
import base64
import hashlib
import json
import requests
import time
import csv
import datetime
import os
import pandas as pd

API_KEY = '0d9fdb6e32d74b9d12e3d894309531838c3aabe8d66b049fd3a7976fbedf2c68'  #@param  {type: "string"}
    
#'to_process_alertIP' + '.csv'
#'to_process_botIP' + '.csv'

def process_iplist(filename_to_process, columnIndex):
    
    with open(filename_to_process + ".csv", newline='') as inputfile:

        if not os.path.exists("downloaded_vtresponse"):
                os.makedirs("downloaded_vtresponse")

        with open(filename_to_process + "_tracker.csv", 'w', newline='') as outputfile:
            ip_list = csv.reader(inputfile, delimiter=',')
            output_writer = csv.writer(outputfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            counter = 0

            for row in ip_list:

                if counter == 0:
                    output_writer.writerow(row + ["Time Run"])
                    outputfile.flush()
                    counter += 1
                    continue

                print("Processing: #" + str(counter) + " - " + row[columnIndex])

                #Get an IP address Report
                r = requests.get("https://www.virustotal.com/api/v3/ip_addresses/"+row[0], headers={"x-apikey":API_KEY})

                # TODO: Make generalised and incorporate timestamp in foldername
                now = datetime.datetime.now()
                dt_string = now.strftime("%d%m%Y")
                
                if not os.path.exists("downloaded_vtresponse/" + dt_string):
                    os.makedirs("downloaded_vtresponse/" + dt_string)
#                 with open("downloaded_vtresponse/" + row[columnIndex] +".json", "w") as outfile:
                with open("downloaded_vtresponse/" + dt_string + "/" + row[columnIndex] +".json", "w") as outfile:

                    outfile.write(r.text)

                output_writer.writerow(row + [datetime.datetime.now()])

                outputfile.flush()

                counter += 1
                time.sleep(16)


def process_json_folder(folder_to_process):
    # Usual Folder: downloaded_vtresponse
    combined_df = pd.DataFrame()
    
    
    for filename in os.listdir(folder_to_process):
        f = os.path.join(folder_to_process, filename)

        # check if it is a file
        if os.path.isfile(f):

            # Opening JSON file
            f = open(f)

            # returns JSON object as
            # a dictionary
            data = json.load(f)

            df_result = pd.json_normalize(data)
            combined_df = pd.concat([combined_df, df_result], ignore_index=True, sort=False)
    
    
    print(combined_df)
    now = datetime.datetime.now()
    dt_string = now.strftime("%d%m%Y")
    
    combined_df.to_csv(folder_to_process + '/' + dt_string + '_parsed-combined.csv')
    


# Process the list of IPs (CSVs ok but IP must be x column in the list)
# process_iplist("Combined_TestKits_BotIPs", 0)
# process_iplist("ip", 0)


# Process the downloaded VT JSONs
# process_json_folder("downloaded_vtresponse_10Jan_combinedFull5k")
process_json_folder("downloaded_vtresponse/23022023")


print("completed")
exit(0)




  data.attributes.regional_internet_registry data.attributes.network  \
0                                       ARIN           34.120.0.0/13   
1                                   RIPE NCC         77.120.224.0/19   
2                                   RIPE NCC          80.253.88.0/21   

  data.attributes.tags data.attributes.country  \
0                   []                      US   
1                   []                      UA   
2                   []                      CH   

   data.attributes.last_analysis_date  data.attributes.as_owner  \
0                        1.676768e+09     GOOGLE-CLOUD-PLATFORM   
1                                 NaN                     Volia   
2                        1.677050e+09  Init7 (Switzerland) Ltd.   

   data.attributes.last_analysis_stats.harmless  \
0                                            87   
1                                            87   
2                                            87   

   data.attributes.last_analysis_sta

In [6]:
print(r.text)

{
    "data": {
        "attributes": {
            "last_dns_records": [
                {
                    "type": "AAAA",
                    "value": "2600:1406:5400:59b::3bd4",
                    "ttl": 20
                },
                {
                    "type": "CNAME",
                    "value": "www.amazon.com.edgekey.net",
                    "ttl": 60
                },
                {
                    "type": "AAAA",
                    "value": "2600:1406:5400:598::3bd4",
                    "ttl": 20
                },
                {
                    "type": "CNAME",
                    "value": "tp.47cf2c8c9-frontier.amazon.com",
                    "ttl": 691
                },
                {
                    "type": "NS",
                    "value": "ns-1144.awsdns-15.org",
                    "ttl": 21600
                },
                {
                    "type": "AAAA",
                    "value": "2600:1406:5400:58e::3bd4",
    